# Twitter user reader - Andy Sharpe, Eric Brito, and Bruce Huang

This program takes a twitter user's ID and lists their most commonly used hashtage, mentinons, and words

1. To do this the twitter library from ipython is used along with a twitter developer account to get a list of the last 200 posts from a single user.
2. If the user inputted is not able to be found or there is not enough alavible data than an appropriate error will be printed
3. For each post in the list each word from the string is extraced and is revoved or modified based on a number of factors including: being an artifact form twitter, not using ascii characters, being a common english word, being a link, and having grammer marks
4. The remaining word strings are then catagorized by their type (word, hashtag or mention) and are added to an array
5. After each post is processed in this way, the lists of word types are put into a function that creates a double (2d) array with the strings as the first value and the number of appearances as the second value.
6. This double array is then sorted by the number of appearances and the top values are printed, but the number of values printed depends on a multiplier for that type (with words printing the lowest fraction and hashtags printing the highest) and it will not print more than 20 of each type to only show the data which is most relevant
7. If there is no data to print for a type after steps 1-5, than an message will be printed instead

In [38]:
#RUNNING THE FOLLOWING MAY TAKE A WHILE, DO NOT RUN THE OTHER CELLS UNTILL THIS IS COMPLETE (and the kernel may need to be restarted)
#this installs the needed twitter library

In [39]:
pip install ipython

Note: you may need to restart the kernel to use updated packages.


In [40]:
#This is the most common 100 words in the english language
normieWords = ["the","of","and","a","to","in","is","you","that","it","he","was","for","on","are","as","with","his","they","i","at","be","this","have","from","or","one","had","by","word","but","not","what","all","were","we","when","your","can","said","there","use","an","each","which","she","do","how","their","if","will","up","other","about","out","many","then","them","these","so","some","her","would","make","like","him","into","time","has","look","two","more","write","go","see","number","no","way","could","people","my","than","first","water","been","call","who","oil","its","now","find","long","down","day","did","get","come","made","may","part"]

In [41]:
#setting up the twitter search abilities
import twitter
api = twitter.Api(consumer_key='dbACwwWs67Xlj8HXtY5wC8c30', consumer_secret='Y2JtxeQAwn5BMUgxAffVfbcfVa9arP8a0qohaLjWJtKkmPmPfK', access_token_key='1129404921340727301-pXeBhw4xYNrscF0ELKnpOn22wS0nkP',access_token_secret='9dNXCwVG9rn3oooq5ZsgCdqM4HXvKvv5avhlMIZ584c3p')

In [57]:
def LookAt(pl,nw): #pl is the list of posts in string format and nw is the list of the most commmon english words
    """Takes a list of strings were each represents the contents of a post and then outputs the most commonly used words, hashtags, and mentions but revoves common words, non-ascii characters, unwanted symbols, and artifacts from twitter"""
    #list of hashtags
    hashes = []
    #list of @s
    ats = []
    #list of all words
    aWords = []
    #looks at each post's content
    for post in pl:
        tempH = []
        tempA = []
        tempRemL = []
        #list of the words in the post
        postWords = list()
        #splits the post into words and removes the most common english words
        for word in post.lower().split():
            if word not in nw:
                postWords.append(word)
        
        #The following code removes/changes/arranges the text in the posts, it has to do things one at a time as there might be repeat errors/problems to deal with
        
        #removes rt and &amp as it shows up on posts and is not an actual "word used"
        for word in postWords:
            if word == 'rt' or word == '&amp;':
                tempRemL.append(word)
        #removes links
        for word in postWords:
            if 'http' in word or 'bit.ly' in word:
                tempRemL.append(word)
        #removes grammar things
        for word in postWords:
            tw = ""
            for ch in word:
                if not (ord(ch) <65 or (ord(ch) >90 and ord(ch) <97) or ord(ch) >122):
                    tw += ch
            if len(tw) == 0:
                tempRemL.append(word)
            else:
                word = tw
        #removes emoji/non-ascii words
        for word in postWords:
            for ch in word:
                if ch > "\x7F":
                    tempRemL.append(word)
                    break
                    
        #removes all of the words in the list that are in the 'removal' list
        for wo in tempRemL:
            if wo in postWords:
                postWords.remove(wo)
            
        #removes symbols
        delSymbolsArr(postWords)
        #removes one letter words
        delOneLen(postWords)

        #adds hashtags to the hashtag list
        for word in postWords:
            if word[0] == '#':
                tempH.append(word)
        #adds '@s' to the mentions list
        for word in postWords:
            if '@' in word or '@' in word:
                tempA.append(word)
        #removes all of the words in the list that are in the mentions and hashtags list and adds them to the total list
        for wo in tempA:
            ats.append(wo)
            if wo in postWords:
                postWords.remove(wo)
        for wo in tempH:
            hashes.append(wo)
            if wo in postWords:
                postWords.remove(wo)
        
        #collects all of the words together -MUST BE LAST
        for word in postWords:
            aWords.append(word)
        
    #prints the most used hashtags
    sortWords(hashes, "hashtags",1)
    #prints the most 'atted' people
    sortWords(ats, "mentions",2)
    #prints the most used words
    sortWords(aWords, "words",4)

In [71]:
def sortWords (swl,typ,division): #swl is the list of words, typ is the name of the word for printing, division is the fraction of numbers to print
    """Takes a list of strings, the name of the string type (EX: hashtag), and what maximum fraction of the list to use, and prints the most occuring strings (that occur more than once) sorted with the number of occurances. If there is repeating strings found or no strings in the first place, an error will be printed instead"""
    #check if the person used any of the words
    if len(swl)>0:
        #code that creates 2d array with the words and the number of them
        hl = []
        for hn1 in range(len(swl)):
            boo = True
            for hn2 in range(len(hl)):
                if swl[hn1] == hl[hn2][0]:
                    boo = False
                    hl[hn2][1]+=1
            if boo:
                hl.append([swl[hn1],1])
        
        #cleans up the list to remove artifacts from the above code
        for hn1 in range(len(hl)):
            if hl[hn1][0] == '':
                hl.pop(hn1)
                break
        #decides on the number of words to print
        hashToP = (len(hl))//division
        #setup for sorting by the second number
        def sortSecond(ppp):
            return ppp[1]
        #sorts in ascending order by the second value in the double arrays
        hl.sort(key = sortSecond, reverse = True)
        #prints the highest used words and their types
        print("\n"+name+"\'s commonly used "+typ+":")
        bole = False
        #if there is over 20 correct words, set it to print 20 max
        if hashToP>20:
            hashToP = 20
        #for loop for printing the words
        for eee in range(hashToP):
            #only prints words that are repeated more than once
            if hl[eee][1] > 1:
                print(hl[eee][0]+" "*(30-len(hl[eee][0]))+"appears "+ str(hl[eee][1]) + " times")
                bole = True
        #this prints if no words were repeated/printed
        if not bole:
            print("-No repeated "+typ+" found")
    else:
        print(name+" has not used any "+typ)

In [72]:
#removes symbols from strings
def delSymbols(string):
    """Takes a string and returns it with all of the unwanted symbols and characters removed"""
    liss = [ord(x) for x in string]
    liss = [y for y in liss if not(y < 35 or (y>35 and y < 64) or (y > 90 and y < 97) or y > 122 or y == ":")]
    return ''.join([chr(c) for c in liss])
#uses the delSymbols function on each string in the array to remove the symbols from the words in it
def delSymbolsArr(arr):
    """Takes a list and calls the delSymbols function on each element, removing symbols from that string in the list"""
    for x in range(len(arr)):
        arr[x] = delSymbols(arr[x])
#removes the one and 0 letter words from the array
def delOneLen(arr):
    """Removes strings from a list that are one or less letters long"""
    for x in arr:
        if len(x) <= 1:
            arr.remove(x)

In [74]:
#RUN THIS CELLS AFTER THE OTHERS FOR THE 'PROGRAM' TO RUN

#gets the id
name = input("Twitter user id")
pCount = 200
postList = []
try :
    #checks if user exists, code will move to 'except' if it does not
    u = api.GetUser(screen_name=name)
    #pCount = int(input("Max number of "+name+"\'s posts to search for"))
    #checks if the number that the user inputed is valid
    if pCount>0:
        #gets list of posts from twitter user
        TempTw = api.GetUserTimeline(screen_name=name, count = pCount)
        print("\n")
        #checks if the user has any posts
        if len(TempTw)>0:
            print(name+"\'s Statistics for their last "+str(len(TempTw))+" posts:\n")
            #turns list of "Status" class into dictionary to be able to use it
            TwList = [i.AsDict() for i in TempTw]
            #takes the dictionary and makes a list of the post's strings out of its
            for z in TwList:
                postList.append(z['text'])
            #runs the main analysis code
            LookAt(postList,normieWords)
        else:
            print(name+" has no posts")
    else:
        print(str(pCount)+" is not a vlaid number")
except:
    print("User with username \""+name+"\" not found")

Twitter user idjackblack


jackblack's Statistics for their last 100 posts:


jackblack's commonly used hashtags:
#postapocalypto               appears 9 times
#jablinskigames               appears 8 times
#tenaciousd                   appears 3 times
#jumanji                      appears 2 times
#kimmel                       appears 2 times
#housewithaclock              appears 2 times

jackblack's commonly used mentions:
@realtenaciousd               appears 17 times
@gassleak                     appears 6 times
@jimmykimmellive              appears 5 times
@jackblack                    appears 4 times
@jimmykimmel                  appears 2 times

jackblack's commonly used words:
new                           appears 10 times
now                           appears 9 times
right                         appears 7 times
rock                          appears 7 times
live                          appears 6 times
postapocalypto                appears 6 times
watch                         a